Plan :

1 - Scipts des API 

2 - API => S3 

3 - S3 => MongoDB 

4 - MongoBD => SQL 

5 - A faire/résoudre

4 - MongoDB => SQL

L'ocjectif de cette étape est de faire passer les données qui sont sous le format json/mongodb à une BDD sql. Pourquoi ?
- Les màj des vols fréquentes s'apparentent à des transactions. Il est plus judicieux d'utiliser SQL lorsqu'il s'agit de transactions.
- La BDD mongodb sera davantage utilisée pour faire des transform sur les documents json. 
- Pour intégrer les données sur dash, il est plus simple d'avoir nos données sous format dataframe. Les dataframes seront plus simples à créer à partir de table sql qui contiennent déjà du lien entre elles. Le dataframe sera directement utilisable sur dash après une simple conversion de la requête sql en dataframe.
- Notre objectif final sera d'afficher simplement les informations d'un vol en cours ou non. Un objcectif d'analyse des données n'est pas encore envisagé mais reste toutefois une possibilité.

In [5]:
! python create_sqlbdd_and_table.py

2022-08-05 14:35:38,117 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-08-05 14:35:38,117 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("airports")
2022-08-05 14:35:38,117 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-05 14:35:38,117 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("airports")
2022-08-05 14:35:38,117 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-05 14:35:38,117 INFO sqlalchemy.engine.Engine 
CREATE TABLE airports (
	iata_code VARCHAR NOT NULL, 
	country_code VARCHAR, 
	icao_code VARCHAR, 
	lat FLOAT, 
	lng FLOAT, 
	name VARCHAR, 
	PRIMARY KEY (iata_code)
)


2022-08-05 14:35:38,117 INFO sqlalchemy.engine.Engine [no key 0.00005s] ()
2022-08-05 14:35:38,118 INFO sqlalchemy.engine.Engine COMMIT
2022-08-05 14:35:38,119 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-08-05 14:35:38,119 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("airports")
2022-08-05 14:35:38,119 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-05 14:35:38,119 INFO 

Le script create_sqlbdd_and_table.py permet de créer la base de données sql nommée "dst-airlines.db" puis de créer les différentes tables qui composeront cette BDD. A savoir :
- Une table airports (iata_code PK)
- Une table countries (code PK)
- Une table cities (city_code PK / country_code FK de la table countries)
- Une table aircraft (hex PK)
- Une table real_time_flights (flight_number PK / dep_iata FK de la table airports / arr_iata FK de la table airports)
- Une table delays_flights (id autoincrementé PK / flight_number_date FK de la table real_time_flights / dep_iata FK de la table airports / arr_iata FK de la table airports)
- Une table test (uniquement pour faire des tests)





In [6]:
! python inject_data.py

Le fichier inject_data.py permet d'automatiser l'injection les données issues de mongodb dans les tables de la BDD sql créée précédemment (fichier create_sqlbdd_and_table.py).
Les principales étapes du script inject_data.py sont les suivantes :
- Importer dans le fichier la BDD mongodb qui  sont stockées sur le cloud
- Importer les différentes collections de la BDD mongodb
- Importer la BDD sql qui a précédemment été créée et qui se nomme "dst_airlines.db"
- Automatiser l'injection de données pour chaque tables

Note : Nous avons un problème de valeur manquantes pour automatiser la création de toutes nos tables (c'est un problème que nous n'avons pas encore réussi à résoudre). Dans ce notebook, nous allons montrer uniquement l'automatisation de création d'une table "test" à partir de champs non manquants. 

In [7]:
! python sql_queries.py

[('0', 7840, 'en-route'), ('1', 2209, 'en-route'), ('2', 10675, 'en-route'), ('3', 10066, 'en-route'), ('4', 12192, 'en-route'), ('5', 10058, 'en-route'), ('6', 9753, 'en-route'), ('7', 10058, 'en-route'), ('8', 10965, 'en-route'), ('9', 7597, 'en-route'), ('10', 3680, 'en-route'), ('11', 11277, 'en-route'), ('12', 9448, 'en-route'), ('13', 11887, 'en-route'), ('14', 10972, 'en-route'), ('15', 11582, 'en-route'), ('16', 10668, 'en-route'), ('17', 10363, 'en-route'), ('18', 10363, 'en-route'), ('19', 12192, 'en-route'), ('20', 10972, 'en-route'), ('21', 10972, 'en-route'), ('22', 10668, 'en-route'), ('23', 10668, 'en-route'), ('24', 10363, 'en-route'), ('25', 10370, 'en-route'), ('26', 10363, 'en-route'), ('27', 10073, 'en-route'), ('28', 10066, 'en-route'), ('29', 9761, 'en-route'), ('30', 9753, 'en-route'), ('31', 7924, 'en-route'), ('32', 6705, 'en-route'), ('33', 5844, 'en-route'), ('34', 0, 'en-route'), ('35', 10972, 'en-route'), ('36', 10668, 'en-route'), ('37', 10363, 'en-route')

Le script sql_queries.py permet de faire des requêtes sql.
En l'occurence, la requête suivante a été effectuées :
"SELECT * FROM test;"

La table test est celle dont les données ont été injectées automatiquement depuis la BDD mongodb. Cela prouve qu'il est possible d'automatiser l'injection des données pour toutes nos tables. Il faut juste que l'on arrive à gérer les valeurs manquantes afin de créer automatiquement toutes nos tables. 

5 - A faire/résoudre

Partie mongobd => sql :
- gérer données manquantes dans les json
- ajout autimatique de la date du jour de la requête dans la table real_time_flight
- voir pour la mise à jour des champs dans la BDD sql
- uen fois qu'on aura toutes nos tables sql : tester la cohérence du lien entre nos table avec des jointures